# Arduino Serial Setups

In [5]:
!pip3 install keyboard

   ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
   --------------------- ------------------ 30.7/58.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 58.1/58.1 kB 1.0 MB/s eta 0:00:00


In [38]:
!pip install pyFirmata2

In [434]:
import serial
import keyboard
import time

In [540]:
ser = serial.Serial(port='COM5', baudrate=115200)
ser.flush()

In [547]:
while True:
    try:
        if keyboard.is_pressed('l'):
            #ser.flush()
            #ser.flushInput()
            ser.reset_input_buffer()
            #time.sleep(0.1)
            input = ser.readline()
            val = str(input, 'UTF-8')
            
            y, p, r, d, t = val.split()
            
            print('yaw: {}, pitch: {}, roll: {}, distance: {}, time: {}'.format(y,p,r,d,t))
            
        if keyboard.is_pressed('q'):
            break
            
        ser.reset_input_buffer()
    except:
        pass

yaw: -21, pitch: -8, roll: -2, distance: 177, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 988, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 988, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 177, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 179, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 85, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 175, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 83, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 174, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 175, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 175, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 988, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 177, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 83, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 172, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 176, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 84, time: 123
yaw: -21, pitch: -8, roll: -2, distance: 82, time: 1

In [549]:
ser.close()

# Arduino Board Emulation

In [1]:
import pyfirmata2

In [85]:
board = pyfirmata2.Arduino('COM4')

In [86]:
#ledPin = board.get_pin('d:13:p')
servo1 = board.get_pin('d:4:s')
servo2 = board.get_pin('d:5:s')
servo3 = board.get_pin('d:6:s')
servo4 = board.get_pin('d:7:s')
servo5 = board.get_pin('d:8:s')
servo6 = board.get_pin('d:9:s')
servo7 = board.get_pin('d:10:s')
servo8 = board.get_pin('d:11:s')
#servo9 = board.get_pin('d:12:s')

In [103]:
servo1.write(70)

In [83]:
board.exit()

# Dependencies

In [105]:
#gym
import gymnasium as gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#helpers
import numpy as np
import random
import os

#stableblines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [106]:
import torch
torch.cuda.is_available()

True

# The Robot Class

In [422]:
class RobotEnv(Env):
    def __init__(self): #sets up the env when first run?
        self.action_space = MultiDiscrete([181,181,181,181,181,181,181,181]) # tap up, down, or no action
        self.observation_space = MultiDiscrete([181,181,181,990,181,181,181,181,181,181,181,181])
        self.state = [90,
                      90,
                      90,
                      100,
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10))]
        self.trial_length = 60
        
    def step(self, action): #

        #set servo values in new observation space
        reward = 0
        for i in range(len(action)):
            self.state[i+4] = action[i]

        #decreate time
        self.trial_length -= 1

        #calculate reward
        reward = 0
        for i in range(3):
            reward += ((abs(90 - self.state[i]) ** 2) * -1)

        if(100 - self.state[3]) < 0:
            reward += ((100 - self.state[3]) ** 2) * -1
        else:
            reward += ((100 - self.state[3]) ** 2)
        
        #conditions for the trial ending
        if self.trial_length <=0:
            done = True
        else:
            done = False
        
        for i in range(3):
            if abs(90 - self.state[i]) >= 30:
                done = True

        info = {}

        #time.sleep(0.01)
        return self.state, reward, done, info

    def render(self): #renders env
        #not going to implement this
        pass
    def reset(self): #resets temp and shower time
        self.state = [60,
                      99,
                      76,
                      100,
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10)),
                      (90 + random.randint(-10,10))]
        self.trial_length = 60
        return self.state

In [424]:
env = RobotEnv()

In [426]:
env.observation_space.sample()

array([ 13, 170,   5, 649, 144, 123,  95, 164,   8,  96,  68, 122],
      dtype=int64)

In [428]:
env.action_space.sample()

array([128, 120, 136, 139,  38, 178,  59,  66], dtype=int64)

In [430]:
env.reset()

[60, 99, 76, 100, 93, 90, 83, 90, 91, 87, 90, 95]

In [432]:
env.step(env.action_space.sample())

([60, 99, 76, 100, 39, 150, 57, 24, 39, 71, 138, 73], -1177, True, {})

# Testing

In [52]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-44
Episode:2 Score:-56
Episode:3 Score:-34
Episode:4 Score:-48
Episode:5 Score:-60


# Training

In [55]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\nurma\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [59]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_14
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.6    |
| time/              |          |
|    fps             | 917      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -25.5        |
| time/                   |              |
|    fps                  | 813          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0062761456 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   

# Save Load Eval

In [64]:
shower_path = os.path.join('Training', 'Saved Models', 'Shower_Model_PPO')

In [66]:
model.save(shower_path)

In [68]:
del model
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [70]:
evaluate_policy(model, env, n_eval_episodes=200)

C:\Users\nurma\anaconda3\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(22.54, 55.328730330633825)